<a href="https://colab.research.google.com/github/rainardw/Moonlight-Diary/blob/main/Moonlight_Diary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# INSTALL (cukup sekali)
!pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import hashlib
import json
import os
import datetime
import getpass  # untuk input password tersembunyi

In [8]:
diary_css = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Caveat:wght@600;700&family=IBM+Plex+Serif:ital@0;1&display=swap');

/* WADAH UTAMA - BUKU DENGAN BACKGROUND PINK */
.diary-book {
    background: linear-gradient(135deg, #f8e9f0 0%, #f4dce5 100%);
    padding: 40px 30px;
    border-radius: 28px;
    color: #333;
    max-width: 950px;
    margin: 30px auto;
    box-shadow: 0 15px 60px rgba(0,0,0,0.1);
    position: relative;
    overflow: hidden;
}

.diary-book::before {
    content: "";
    position: absolute;
    top: 0; left: 0; right: 0; bottom: 0;
    background:
        url('data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHdpZHRoPSIxMDAlIiBoZWlnaHQ9IjEwMCUiPgo8cmVjdCB3aWR0aD0iMTAwJSIgaGVpZ2h0PSIxMDAlIiBmaWxsPSJub25lIi8+CjxyZWN0IHdpZHRoPSIxMDAlIiBoZWlnaHQ9IjEwMCUiIGZpbGw9InVybCgjZ3JhZCkiLz4KPGcgb3BhY2l0eT0iMC4xIj4KPHBhdGggZD0iTTEwLDIwIEwxNSwyNSBMNTAsMjAgTDQ1LDE1IEwxMCwyMCIgc3Ryb2tlPSIjZmYyMjIyIiBzdHJva2Utd2lkdGg9IjIiIGZpbGw9Im5vbmUiLz4KPHBhdGggZD0iTTEwLDUwIEwxNSw1NSBMNTAsNTAgTDQ1LDQ1IEwxMCw1MCIgc3Ryb2tlPSIjZmYyMjIyIiBzdHJva2Utd2lkdGg9IjIiIGZpbGw9Im5vbmUiLz4KPHBhdGggZD0iTTEwLDgwIEwxNSw4NSBMNTAsODAgTDQ1LDc1IEwxMCw4MCIgc3Ryb2tlPSIjZmYyMjIyIiBzdHJva2Utd2lkdGg9IjIiIGZpbGw9Im5vbmUiLz4KPC9nPgo8ZGVmcz4KPGxpbmVhckdyYWRpZW50IGlkPSJncmFkIiB4MT0iMCUiIHkxPSIwJSIgeDI9IjEwMCUiIHkyPSIxMDAlIj4KPHN0b3Agb2Zmc2V0PSIwJSIgc3RvcC1jb2xvcj0iI2Y4ZTlmMCIvPgk8c3RvcCBvZmZzZXQ9IjEwMCUiIHN0b3AtY29sb3I9IiNmNGRjZTUifD4KPC9saW5lYXJHcmFkaWVudD4KPC9kZWZzPgo8L3N2Zz4=') repeat;
    pointer-events: none;
}

.book-cover {
    background: linear-gradient(135deg, #ff9aa2, #ffb7b2);
    padding: 30px;
    border-radius: 20px 20px 0 0;
    text-align: center;
    position: relative;
    z-index: 10;
    box-shadow: inset 0 0 20px rgba(0,0,0,0.05);
}

.book-title {
    font-family: 'Caveat', cursive;
    font-size: 3.5em;
    color: #fff;
    text-shadow: 0 2px 8px rgba(0,0,0,0.2);
    letter-spacing: 2px;
    margin-bottom: 10px;
}

.book-subtitle {
    font-family: 'IBM Plex Serif', serif;
    font-size: 1.2em;
    color: rgba(255,255,255,0.8);
    font-style: italic;
    margin-bottom: 20px;
}

.book-page {
    background:
        linear-gradient(to bottom,
            rgba(255,255,255,0.98) 0%,
            rgba(255,250,250,0.98) 100%),
        repeating-linear-gradient(
            to bottom,
            transparent,
            transparent 29px,
            #fff 29px,
            #fff 30px
        );
    color: #333;
    padding: 40px;
    border-radius: 0 0 20px 20px;
    min-height: 350px;
    box-shadow:
        inset 0 0 15px rgba(0,0,0,0.05),
        0 10px 30px rgba(0,0,0,0.05);
    font-family: 'IBM Plex Serif', serif;
    font-size: 18px;
    line-height: 1.8;
    position: relative;
    transition: all 0.4s ease;
}

.book-page::before {
    content: "";
    position: absolute;
    top: 0; left: 0; right: 0; bottom: 0;
    background:
        radial-gradient(circle at 20% 20%, rgba(255,255,255,0.1) 0%, transparent 20%),
        radial-gradient(circle at 80% 80%, rgba(255,255,255,0.1) 0%, transparent 20%);
    pointer-events: none;
}

.lock-screen {
    position: absolute;
    top: 0; left: 0; right: 0; bottom: 0;
    background: rgba(255,255,255,0.9);
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    z-index: 100;
    border-radius: 20px;
    backdrop-filter: blur(5px);
}

.lock-icon {
    font-size: 6em;
    color: #ff6b6b;
    margin-bottom: 20px;
    animation: shake 2s infinite ease-in-out;
}

@keyframes shake {
    0% { transform: translateX(0); }
    25% { transform: translateX(-5px); }
    50% { transform: translateX(5px); }
    75% { transform: translateX(-5px); }
    100% { transform: translateX(0); }
}

.lock-text {
    font-family: 'Caveat', cursive;
    font-size: 1.8em;
    color: #888;
    text-align: center;
    margin: 20px 0;
}

.lock-hint {
    font-size: 0.9em;
    color: #aaa;
    font-style: italic;
}

.write-area {
    width: 100%;
    height: 250px;
    padding: 15px;
    border: 2px solid #ffdde1;
    border-radius: 12px;
    font-family: 'IBM Plex Serif', serif;
    font-size: 16px;
    line-height: 1.6;
    resize: vertical;
    background: white;
    box-shadow: 0 4px 12px rgba(0,0,0,0.05);
    transition: all 0.3s ease;
}

.write-area:focus {
    outline: none;
    border-color: #ff6b6b;
    box-shadow: 0 0 15px rgba(255,107,107,0.2);
}

.book-btn {
    margin: 0 10px;
    padding: 12px 28px;
    border: none;
    border-radius: 50px;
    font-weight: bold;
    cursor: pointer;
    transition: all 0.35s ease;
    color: white;
    font-size: 16px;
    box-shadow: 0 4px 12px rgba(0,0,0,0.1);
    backdrop-filter: blur(4px);
}

.book-btn:hover {
    transform: translateY(-3px) scale(1.03);
    box-shadow: 0 6px 18px rgba(0,0,0,0.15);
}

.book-btn:active {
    transform: translateY(-1px);
}

.btn-write { background: linear-gradient(45deg, #ff9aa2, #ffb7b2); }
.btn-view  { background: linear-gradient(45deg, #ff6b6b, #ff4b2b); }
.btn-lock  { background: linear-gradient(45deg, #ff9aa2, #ff6b6b); }

.entry-list {
    background: rgba(255, 255, 255, 0.8);
    border: 1px solid #ffdde1;
    border-radius: 12px;
    padding: 20px;
    margin-top: 20px;
    max-height: 350px;
    overflow-y: auto;
    font-family: 'IBM Plex Serif', serif;
    box-shadow: inset 0 0 8px rgba(0,0,0,0.05);
}

.entry-item {
    padding: 12px 0;
    border-bottom: 1px dashed #eee;
}
.entry-item:last-child {
    border-bottom: none;
}
.entry-date {
    font-weight: bold;
    color: #ff6b6b;
    font-size: 0.95em;
}
.entry-preview {
    margin-top: 5px;
    color: #333;
    font-style: italic;
}

.msg-box {
    text-align: center;
    padding: 12px;
    margin-top: 15px;
    border-radius: 10px;
    font-weight: 500;
}
.msg-success { background: rgba(255, 183, 178, 0.3); color: #ff6b6b; }
.msg-error { background: rgba(255, 107, 107, 0.3); color: #c0392b; }
.msg-info { background: rgba(255, 183, 178, 0.3); color: #2980b9; }

</style>
"""
display(HTML(diary_css))

# === STATE ===
DIARY_FILE = "my_diary.json"
password_hash = None
entries = []
unlocked = False

# === FUNGSI UTAMA ===
def load_diary():
    global password_hash, entries
    if os.path.exists(DIARY_FILE):
        with open(DIARY_FILE, 'r') as f:
            data = json.load(f)
            password_hash = data.get("password_hash")
            entries = data.get("entries", [])
    else:
        password_hash = None
        entries = []

def save_diary():
    with open(DIARY_FILE, 'w') as f:
        json.dump({
            "password_hash": password_hash,
            "entries": entries
        }, f)

def set_password(pwd):
    global password_hash
    password_hash = hashlib.sha256(pwd.encode()).hexdigest()
    save_diary()

def verify_password(pwd):
    if not password_hash:
        return False
    return hashlib.sha256(pwd.encode()).hexdigest() == password_hash

# === WIDGETS ===
cover = widgets.HTML('<div class="book-cover"><div class="book-title">🌸 Moonlight Journal</div><div class="book-subtitle">Tulis rahasia hatimu di atas kertas ini.</div></div>')
page = widgets.HTML('')
lock_screen = widgets.HTML('')
write_area = widgets.Textarea(
    placeholder="Tulis pikiranmu di sini...\nBiarkan jiwa mengalir di atas kertas malam ini.",
    layout=widgets.Layout(width='100%', height='250px')
)
write_area.disabled = True
write_btn = widgets.Button(description='📝 Tulis Entri', button_style='info')
view_btn = widgets.Button(description='👁️ Lihat Entri', button_style='warning')
lock_btn = widgets.Button(description='🔒 Buka Kunci', button_style='success')
save_btn = widgets.Button(description='💾 Simpan')

pwd_input = widgets.Password(value='', placeholder='Masukkan password', layout=widgets.Layout(width='300px'))
pwd_submit = widgets.Button(description='🔓 Buka Kunci', button_style='success')
output_area = widgets.Output()

showing_password_input = False

def render_book():
    global unlocked
    if unlocked:
        today = datetime.datetime.now().strftime("%A, %d %B %Y")
        page.value = f'''
        <div class="book-page">
            <div style="text-align:center; color:#ff6b6b; font-family:'Caveat', cursive; font-size:1.8em; margin-bottom:20px;">
                {today}
            </div>
            <div id="content-area">{write_area.value}</div>
        </div>
        '''
        lock_screen.value = ""
        write_area.disabled = False
        lock_btn.description = '🔐 Kunci'
        # Sembunyikan input
        pwd_input.layout.display = 'none'
        pwd_submit.layout.display = 'none'
    else:
        lock_screen.value = '''
        <div class="lock-screen">
            <div class="lock-icon">🔒</div>
            <div class="lock-text">Jurnal ini terkunci.</div>
            <div class="lock-hint">Buka dengan password untuk menulis.</div>
        </div>
        '''
        page.value = '<div class="book-page"></div>'
        write_area.disabled = True
        lock_btn.description = '🔒 Buka Kunci'
        if showing_password_input:
            pwd_input.layout.display = 'block'
            pwd_submit.layout.display = 'block'
        else:
            pwd_input.layout.display = 'none'
            pwd_submit.layout.display = 'none'

def show_message(msg, msg_type="info"):
    cls = "msg-" + msg_type
    with output_area:
        clear_output()
        display(HTML(f'<div class="msg-box {cls}">{msg}</div>'))

def on_lock_click(b):
    global unlocked, showing_password_input
    load_diary()
    if unlocked:
        unlocked = False
        render_book()
        show_message("Jurnal telah dikunci.", "info")
        showing_password_input = False
    else:
        pwd_input.value = ''
        showing_password_input = True
        render_book()

# ✨ INI YANG DIPERBAIKI: tambahkan 'global unlocked'
def on_pwd_submit_click(b):
    global unlocked  # <<<< INI KUNCI UTAMA!
    pwd = pwd_input.value.strip()
    if not pwd:
        show_message("Password tidak boleh kosong.", "error")
        return
    if not password_hash:
        # Set new password
        set_password(pwd)
        unlocked = True  # <<<< UPDATE STATUS!
        render_book()
        show_message("Password disetel! Selamat menulis.", "success")
    else:
        if verify_password(pwd):
            unlocked = True  # <<<< UPDATE STATUS!
            render_book()
            show_message("Selamat datang kembali. Duniamu menunggu.", "success")
        else:
            show_message("❌ Password salah.", "error")

def on_write_click(b):
    global unlocked
    if not unlocked:
        show_message("🔒 Buka kunci jurnal dulu!", "error")
        return
    content = write_area.value.strip()
    if not content:
        show_message("Tulis sesuatu dulu, sayang.", "error")
        return
    entry = {
        "date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
        "content": content
    }
    entries.append(entry)
    save_diary()
    write_area.value = ""
    show_message("✅ Entri tersimpan di kertas malam ini.", "success")

def on_view_click(b):
    global unlocked
    if not unlocked:
        show_message("🔒 Buka kunci jurnal dulu!", "error")
        return
    if not entries:
        show_message("Belum ada entri. Mulailah menulis hari ini.", "info")
        return
    html = '<div class="entry-list">'
    for entry in reversed(entries[-7:]):
        date = entry["date"]
        preview = entry["content"].replace('\n', ' ')[:200]
        if len(entry["content"]) > 200:
            preview += "..."
        html += f'<div class="entry-item"><div class="entry-date">{date}</div><div class="entry-preview">"{preview}"</div></div>'
    html += '</div>'
    with output_area:
        clear_output()
        display(HTML(html))

# === HUBUNGKAN ===
pwd_submit.on_click(on_pwd_submit_click)
write_btn.on_click(on_write_click)
view_btn.on_click(on_view_click)
lock_btn.on_click(on_lock_click)

# === RENDER AWAL ===
load_diary()
render_book()

# === TAMPILKAN SEMUA ===
ui = widgets.VBox([
    cover,
    page,
    lock_screen,
    write_area,
    widgets.HBox([write_btn, view_btn, lock_btn], layout=widgets.Layout(justify_content='center')),
    pwd_input,
    pwd_submit,
    output_area
])

display(ui)

# Tambahkan petal bunga
display(HTML('''
<div style="position:absolute; top:10%; left:5%; opacity:0.3; font-size:2em; animation:float 8s infinite;">🌸</div>
<div style="position:absolute; top:20%; right:10%; opacity:0.3; font-size:1.5em; animation:float 8s infinite; animation-delay:2s;">🌺</div>
<div style="position:absolute; bottom:15%; left:15%; opacity:0.3; font-size:1.8em; animation:float 8s infinite; animation-delay:4s;">🌼</div>
<div style="position:absolute; bottom:25%; right:5%; opacity:0.3; font-size:1.2em; animation:float 8s infinite; animation-delay:6s;">🌷</div>
<style>
@keyframes float {
    0% { transform: translate(0,0) rotate(0deg); }
    50% { transform: translate(-10px,10px) rotate(10deg); }
    100% { transform: translate(0,0) rotate(0deg); }
}
</style>
'''))